## Bayesiansk optimering 

**Hvordan får vi mest mulig informasjon ut av et eksperiment med færrest mulig målinger? Dette er et omfattende og komplisert problem, men vi skal i denne notebooken lære å bruke en metode som har gitt lovende resultater de siste årene.**

Moderne maskinlærings(ML)-teknikker er gjerne basert på store **datasett**, bestående av "inn-data" i form av **målinger** (innen ML typisk kalt "features") med tilhørende "ut-data" i form av **resultater** (innen ML gjerne kalt "labels" eller "targets"). Innen den ML-teknikken som kalles "supervised learning" er målet å trene en **modell** så den kan gjøre **prediksjoner** i samsvar med datasettet. Du har kanskje vært borti dette før gjennom lineær regresjon, hvor modellen er et polynom.

<div>
    <img src="regression_model_3.png" width = 60%>
    <center>Maskinlæring: et datasett med målinger og resultater brukes for å lære opp en modell på datamaskinen.</center>
</div>

Eksperimenter kan imidlertid være tids- og arbeidskrevende, og vi har ikke alltid tilgang på store datasett. I tillegg kan det være en utfordring at vi ikke har forutsetning for å anta noe om den matematiske formen på resultatene. I slike tilfeller passer det å benytte noe som kalles **Gaussiske prosesser** (GP) som modell. Om vi benytter GP for å predikere et maksimum (topp- eller bunnpunkt) i målingene kalles dette for **Bayesiansk optimering** (BO). En fordel med denne metoden er at den kan ta måleusikkerhet med i betraktningen.

I denne notebooken vil du lære deg å bruke Python-modulen <a href="">B-tjenesten/xdai</a> til å
1. Planlegge målingene dine. (eksperimentell design)
2. Tilpasse en Gaussisk prosess regressor (GPR) til datasettet ditt. (regresjon)
3. Predikere en måling for optimalt resultat. (Bayesiansk optimering)
4. Fortolke og fremstille resultatene visuelt.

Målet er å gjøre deg i stand til å finne optimale målepunkter for et bredt utvalg problemstillinger. Husk imidlertid at maskinlæringsmetoder ikke kan bli bedre enn de datasettene vi mater algoritmen med, så godt håndtverk og kritisk sans er avgjørende for å oppnå gode resultater.

<div class="alert alert-block alert-info">
    <h2> Diskusjonsoppgaver</h2>
<ul>    
 <li>Gi 3 eksempler på "inn-data" og "ut-data". </li>
<li>Hva menes med "å predikere et maksimum i målingene"? </li>
<li>Hva innebærer det å optimere noe? Kan du gi noen eksempler på optimering generelt?</li>
<li>Kan du tenke deg noen tilfeller hvor du kan anta noe om formen på den matematiske formen på resultatene?</li>
<li>Kan du tenke deg noen tilfeller hvor du ikke kan anta noe om denne formen?</li>
<li>Hva legger du i å "lære opp en modell på datamaskinen"? Hvordan tror du dette gjøres i praksis?</li>
    </ul>
</div>


In [8]:
import xdai 
import numpy as np

In [7]:
import numpy as np


def visualize_doe_grid(doe_grid_):
    import evince as ev
    
    class doe_box():
        def __init__(self, doe_grid_):
            self.pos = np.array(doe_grid_[:, 1:].T, dtype = np.float32)
            self.n_bubbles = self.pos.shape[1]
            self.size = np.max(self.pos, axis = 1)
            self.masses = np.ones(self.pos.shape[1])*.1
        
    return ev.MDView(doe_box(doe_grid_))


doe_grid_ = xdai.designer.doe_grid([[-1,1], [-1.4,1.4], [-1.8, 1.8]], design = 2)

visualize_doe_grid(doe_grid_)

MDView(bg_color=[1.0, 1.0, 1.0], box=[1.399999976158142, 1.7999999523162842], colors=b'\xdc8\xb3>\xdb\xb9m?m\x…

In [9]:
all_y = np.random.uniform(-1,1,grid.shape[0])
all_x = grid
gprr = xdai.gpr.Regressor(all_x, all_y, measurement_standard_deviation=0.1)
gprr.predict(np.random.uniform(-1,1,3))

NameError: name 'grid' is not defined

## 1. Planlegg og gjennomfør målingene

Anta at du ønsker å finne en optimal sammensetning av målevariablene $\mathbf{x}_*$ som inngår i et eksperiment, for eksempel temperatur, trykk, konsentrasjoner, varighet og lignende. Anta også at det tar litt tid å gjennomføre hvert eksperiment, så du vil unngå å gjøre for mange.

Det et er lite effektivt (og kanskje umulig) å sjekke *alle mulige målinger* $\{\mathbf{x}\}$. Det vi heller ønsker er å gjøre noen få målinger, og basert på disse lage oss et kart over hvor den optimale målingen mest sannsynlig befinner seg. 

Vi trenger i dette tilfellet å planlegge et sett målinger som vi håper at vil gi oss best mulig informasjon om hvordan systemet oppfører seg. Slik planlegging kalles **eksperimentell design**. Sannsynligvis har du noen forkunnskaper om hvor den optimale målingen befinner seg, så det er hensiktsmessig å velge et målområde avgrenset av nedre og øvre terskelverdier i målevariablene. 

I en dimensjon er dette enkelt. Forestill deg for eksempel at du skal finne en optimal temperatur for veksten av en bakteriekultur. Du antar at bakteriene trives best ved romtemperatur, så du velger et måleintervall mellom 10 og 30 grader Celsius. Deretter gjør du målinger av vekstraten for 10 grader, 30 grader og kanskje for 20 grader (i midten av intervallet ditt). 

Det finnes flere ulike teknikker for å designe et eksperiment (se for eksempel <a href="https://www.itl.nist.gov/div898/handbook/pri/section5/pri5.htm">her</a>). Vi skal benytte det som kalles et Box-Behnken design [] som minner om eksempelet i en dimensjon over, bare at det gjøres i tre dimensjoner.

In [8]:
målinger = doe_grid([[-1,1], [-1,1], [-1,1]], design = 2)

html_table(målinger)

,1.0,0.0,0.0,0.0
,2.0,0.0,0.0,0.0
,3.0,0.0,0.0,0.0
,4.0,-1.0,-1.0,0.0
,5.0,0.0,-1.0,-1.0
,6.0,0.0,-1.0,1.0
,7.0,1.0,-1.0,0.0
,8.0,-1.0,0.0,-1.0
,9.0,-1.0,0.0,1.0
,10.0,1.0,0.0,-1.0
,11.0,1.0,0.0,1.0


In [ ]:
visualize_doe_grid()

## 2. Regresjon


